In [1]:
# !pip install sagemaker==1.38.6

In [2]:
from sagemaker import get_execution_role
from sagemaker.session import Session
from sagemaker.tensorflow import TensorFlow

In [3]:
import boto3
import random
import json
import importlib
import bidding_data
import config

In [4]:
bidding_data.download_stats_file_from_s3()

In [5]:
# S3 bucket for saving code and model artifacts.
# Feel free to specify a different bucket here if you wish.
bucket = 'wsbidder'

# Location where results of model training are saved.
output_path = 's3://{}/trainer_predict_click/model_output'.format(bucket)
model_dir = 's3://{}/trainer_predict_click/model'.format(bucket)

# train_instance_type = 'ml.c4.xlarge'
train_instance_type = 'ml.p2.xlarge'
train_instance_count = 1
hyperparameters = {'epochs': 60000, 'batch_size': 512, 'config_file': 'config.json'}
inputs = {'train': 's3://wsbidder/trainer_predict_click/data/train/',
          'eval': 's3://wsbidder/trainer_predict_click/data/eval/',
          'test': 's3://wsbidder/trainer_predict_click/data/eval/'}
# inputs = {'train': '/opt/ml/input/data/train/',
#           'eval': '/opt/ml/input/data/eval/',
#           'test': '/opt/ml/input/data/eval/'}
base_job_name = 'tf-click-prediction'

In [6]:
estimator = TensorFlow(entry_point='train_predict_click.py',
                       source_dir='.',
                       output_path=output_path,
                       model_dir=model_dir,
                       train_instance_type=train_instance_type,
                       train_instance_count=train_instance_count,
                       hyperparameters=hyperparameters,
                       role=get_execution_role(),
#                        image_name='520713654638.dkr.ecr.eu-west-1.amazonaws.com/sagemaker-tensorflow-scriptmode:1.12.0-gpu-py3',
                       base_job_name=base_job_name,
                       framework_version='1.14.0',
                       py_version='py3',
#                        distributions={'parameter_server': {'enabled': True}},
                       script_mode=True)

In [7]:
# from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner

# # Define loss objective
# loss_objective_metric_name = 'loss'
# loss_objective_type = 'Minimize'
# loss_metric_definitions = [{'Name': 'loss',
#                        'Regex': 'loss = ([0-9\\.]+)'}]

# # acc_objective_metric_name = 'accuracy'
# # acc_objective_type = 'Maximize'
# # acc_metric_definitions = [{'Name': 'accuracy',
# #                        'Regex': 'accuracy = ([0-9\\.]+)'}]

# # Define hyperparameter ranges
# hyperparameter_ranges = {
#                             'learning_rate': ContinuousParameter(1e-6, 1e-4),
#                             'dropout_rate': ContinuousParameter(0.0, 0.5),
#                             'layer_number': IntegerParameter(4, 12),
#                             'node_number': IntegerParameter(128, 512)
#                         }  
# # hyperparameter_ranges = {
# #                             'learning_rate': ContinuousParameter(0.0000001, 0.0001),
# #                             'dropout_rate': ContinuousParameter(0.0, 0.9),
# #                             'batch_size': IntegerParameter(512, 4096)
# #                         }  
# # Initialise Sagemaker's hyperparametertuner
# tuner = HyperparameterTuner(estimator,
#                             loss_objective_metric_name,
#                             hyperparameter_ranges,
#                             loss_metric_definitions,
#                             max_jobs=16,
#                             max_parallel_jobs=4,
#                             objective_type=loss_objective_type)

In [8]:
s3 = boto3.resource('s3')
for obj in s3.Bucket(bucket).objects.filter(Prefix='trainer_predict_click/model/'):
    s3.Object(bucket,obj.key).delete()

In [9]:
# tuner.fit(inputs)

In [10]:
# tuner.deploy(
#     initial_instance_count=1,
#     instance_type='ml.c5.large',
#     endpoint_name='ImpressionPredictionProductionEndpoint',
#     update_endpoint=True
# )

In [11]:
# def update_model(model_data_url):
#     from sagemaker.tensorflow.serving import Model
#     from sagemaker import get_execution_role

#     model_data = model_data_url

#     model = Model(
#         model_data=model_data, 
#         role=get_execution_role(),
#         image='763104351884.dkr.ecr.eu-west-1.amazonaws.com/tensorflow-inference:1.14-cpu',
#         framework_version='1.14.0',
#         env= {
#             "SAGEMAKER_TFS_ENABLE_BATCHING":"true",
#             "SAGEMAKER_TFS_MAX_BATCH_SIZE": "32",
#             "SAGEMAKER_TFS_BATCH_TIMEOUT_MICROS":"10000"
#           }
#     )
#     model.deploy(
#         initial_instance_count=1,
#         instance_type='ml.c5.large',
#         update_endpoint=True,
#         endpoint_name='ClickPredictionProductionEndpoint'
#     )
# model_data_url = 's3://wsbidder/trainer_predict_click/model_output/tf-click-prediction-2020-03-02-13-55-18-155/output/model.tar.gz'
# update_model(model_data_url)

In [ ]:
%%time

estimator.fit(inputs)

2020-03-06 14:09:02 Starting - Starting the training job...
2020-03-06 14:09:04 Starting - Launching requested ML instances......
2020-03-06 14:10:04 Starting - Preparing the instances for training......
2020-03-06 14:11:21 Downloading - Downloading input data.........
2020-03-06 14:12:53 Training - Downloading the training image../usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518


2020-03-06 14:13:13 Training - Training image download completed. Training in progress.I0306 14:13:22.229683 140635362445056 estimator.py:209] Using config: {'_model_dir': 's3://wsbidder/trainer_predict_click/model/1583504001/', '_tf_random_seed': 42, '_save_summary_steps': 100, '_save_checkpoints_steps': 100, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fe7dd998320>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0

W0306 14:13:51.268481 140635362445056 metrics_impl.py:804] Trapezoidal rule is known to produce incorrect PR-AUCs; please switch to "careful_interpolation" instead.
W0306 14:13:51.294061 140635362445056 metrics_impl.py:804] Trapezoidal rule is known to produce incorrect PR-AUCs; please switch to "careful_interpolation" instead.
I0306 14:13:51.319001 140635362445056 estimator.py:1147] Done calling model_fn.
I0306 14:13:51.341837 140635362445056 evaluation.py:255] Starting evaluation at 2020-03-06T14:13:51Z
I0306 14:13:51.704108 140635362445056 monitored_session.py:240] Graph was finalized.
W0306 14:13:51.706790 140635362445056 deprecation.py:323] From /usr/local/lib/python3.6/site-packages/tensorflow/python/training/saver.py:1282: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
I0306 14:13:51.776081 140635362445056 saver.

I0306 14:14:29.582844 140635362445056 evaluation.py:167] Evaluation [80/100]
I0306 14:14:29.991272 140635362445056 evaluation.py:167] Evaluation [90/100]
I0306 14:14:30.439348 140635362445056 evaluation.py:167] Evaluation [100/100]
I0306 14:14:30.600344 140635362445056 evaluation.py:275] Finished evaluation at 2020-03-06-14:14:30
I0306 14:14:30.600638 140635362445056 estimator.py:2039] Saving dict for global step 300: accuracy = 0.5077053, accuracy_baseline = 0.5196947, auc = 0.5466728, auc_precision_recall = 0.51954764, average_loss = 0.69263935, global_step = 300, label/mean = 0.5196947, loss = 369.12357, precision = 0.5927035, prediction/mean = 0.49209026, recall = 0.16853933
I0306 14:14:30.702493 140635362445056 estimator.py:2099] Saving 'checkpoint_path' summary for global step 300: s3://wsbidder/trainer_predict_click/model/1583504001/model.ckpt-300
I0306 14:14:30.818808 140635362445056 exporter.py:299] Performing best model export.
I0306 14:14:31.079579 140635362445056 estimator.

I0306 14:15:10.568363 140635362445056 evaluation.py:167] Evaluation [60/100]
I0306 14:15:10.976463 140635362445056 evaluation.py:167] Evaluation [70/100]
I0306 14:15:11.402561 140635362445056 evaluation.py:167] Evaluation [80/100]
I0306 14:15:11.809691 140635362445056 evaluation.py:167] Evaluation [90/100]
I0306 14:15:12.210262 140635362445056 evaluation.py:167] Evaluation [100/100]
I0306 14:15:12.376183 140635362445056 evaluation.py:275] Finished evaluation at 2020-03-06-14:15:12
I0306 14:15:12.376457 140635362445056 estimator.py:2039] Saving dict for global step 500: accuracy = 0.51463705, accuracy_baseline = 0.5196947, auc = 0.5500448, auc_precision_recall = 0.54529655, average_loss = 0.6921595, global_step = 500, label/mean = 0.5196947, loss = 368.8678, precision = 0.5771148, prediction/mean = 0.49142548, recall = 0.24719101
I0306 14:15:12.505482 140635362445056 estimator.py:2099] Saving 'checkpoint_path' summary for global step 500: s3://wsbidder/trainer_predict_click/model/158350

...

I0306 15:10:16.332762 140635362445056 basic_session_run_hooks.py:606] Saving checkpoints for 19300 into s3://wsbidder/trainer_predict_click/model/1583504001/model.ckpt.
I0306 15:10:18.265333 140635362445056 estimator.py:1145] Calling model_fn.
W0306 15:10:20.874880 140635362445056 metrics_impl.py:804] Trapezoidal rule is known to produce incorrect PR-AUCs; please switch to "careful_interpolation" instead.
W0306 15:10:20.900089 140635362445056 metrics_impl.py:804] Trapezoidal rule is known to produce incorrect PR-AUCs; please switch to "careful_interpolation" instead.
I0306 15:10:20.924550 140635362445056 estimator.py:1147] Done calling model_fn.
I0306 15:10:20.947315 140635362445056 evaluation.py:255] Starting evaluation at 2020-03-06T15:10:20Z
I0306 15:10:21.306507 140635362445056 monitored_session.py:240] Graph was finalized.
I0306 15:10:21.403523 140635362445056 saver.py:1286] Restoring parameters from s3://wsbidder/trainer_predict_click/model/1583504001/model.ckpt-19300
I0306 15:10

I0306 15:11:06.471074 140635362445056 basic_session_run_hooks.py:606] Saving checkpoints for 19600 into s3://wsbidder/trainer_predict_click/model/1583504001/model.ckpt.
I0306 15:11:08.497398 140635362445056 estimator.py:1145] Calling model_fn.
W0306 15:11:11.081217 140635362445056 metrics_impl.py:804] Trapezoidal rule is known to produce incorrect PR-AUCs; please switch to "careful_interpolation" instead.
W0306 15:11:11.106586 140635362445056 metrics_impl.py:804] Trapezoidal rule is known to produce incorrect PR-AUCs; please switch to "careful_interpolation" instead.
I0306 15:11:11.130990 140635362445056 estimator.py:1147] Done calling model_fn.
I0306 15:11:11.153914 140635362445056 evaluation.py:255] Starting evaluation at 2020-03-06T15:11:11Z
I0306 15:11:11.513153 140635362445056 monitored_session.py:240] Graph was finalized.
I0306 15:11:11.607244 140635362445056 saver.py:1286] Restoring parameters from s3://wsbidder/trainer_predict_click/model/1583504001/model.ckpt-19600
I0306 15:11

I0306 15:11:56.270926 140635362445056 basic_session_run_hooks.py:606] Saving checkpoints for 19900 into s3://wsbidder/trainer_predict_click/model/1583504001/model.ckpt.
I0306 15:11:58.200276 140635362445056 estimator.py:1145] Calling model_fn.
W0306 15:12:00.883527 140635362445056 metrics_impl.py:804] Trapezoidal rule is known to produce incorrect PR-AUCs; please switch to "careful_interpolation" instead.
W0306 15:12:00.908700 140635362445056 metrics_impl.py:804] Trapezoidal rule is known to produce incorrect PR-AUCs; please switch to "careful_interpolation" instead.
I0306 15:12:00.933002 140635362445056 estimator.py:1147] Done calling model_fn.
I0306 15:12:00.955879 140635362445056 evaluation.py:255] Starting evaluation at 2020-03-06T15:12:00Z
I0306 15:12:01.324201 140635362445056 monitored_session.py:240] Graph was finalized.
I0306 15:12:01.417662 140635362445056 saver.py:1286] Restoring parameters from s3://wsbidder/trainer_predict_click/model/1583504001/model.ckpt-19900
I0306 15:12

I0306 15:12:44.529219 140635362445056 builder_impl.py:421] SavedModel written to: s3://wsbidder/trainer_predict_click/model/1583504001/export/predict_imp/temp-b'1583507560'/saved_model.pb
I0306 15:12:45.598366 140635362445056 basic_session_run_hooks.py:692] global_step/sec: 4.78055
I0306 15:12:45.601300 140635362445056 basic_session_run_hooks.py:260] loss = 184.33347, step = 20100 (20.919 sec)
I0306 15:12:50.539656 140635362445056 basic_session_run_hooks.py:606] Saving checkpoints for 20200 into s3://wsbidder/trainer_predict_click/model/1583504001/model.ckpt.
I0306 15:12:52.546552 140635362445056 estimator.py:1145] Calling model_fn.
W0306 15:12:55.185921 140635362445056 metrics_impl.py:804] Trapezoidal rule is known to produce incorrect PR-AUCs; please switch to "careful_interpolation" instead.
W0306 15:12:55.211079 140635362445056 metrics_impl.py:804] Trapezoidal rule is known to produce incorrect PR-AUCs; please switch to "careful_interpolation" instead.
I0306 15:12:55.235598 1406353

I0306 15:13:34.378720 140635362445056 evaluation.py:167] Evaluation [70/100]
I0306 15:13:34.786953 140635362445056 evaluation.py:167] Evaluation [80/100]
I0306 15:13:35.204878 140635362445056 evaluation.py:167] Evaluation [90/100]
I0306 15:13:35.616714 140635362445056 evaluation.py:167] Evaluation [100/100]
I0306 15:13:35.778416 140635362445056 evaluation.py:275] Finished evaluation at 2020-03-06-15:13:35
I0306 15:13:35.778688 140635362445056 estimator.py:2039] Saving dict for global step 20400: accuracy = 0.59216183, accuracy_baseline = 0.5196947, auc = 0.6455047, auc_precision_recall = 0.65663075, average_loss = 0.65573627, global_step = 20400, label/mean = 0.5196947, loss = 349.45703, precision = 0.6102882, prediction/mean = 0.5115043, recall = 0.59550565
I0306 15:13:35.871365 140635362445056 estimator.py:2099] Saving 'checkpoint_path' summary for global step 20400: s3://wsbidder/trainer_predict_click/model/1583504001/model.ckpt-20400
I0306 15:13:36.060650 140635362445056 basic_sess

In [ ]:
# estimator.deploy(
#     initial_instance_count=1,
#     instance_type='ml.c5.large',
#     endpoint_name='ClickPredictionProductionEndpoint'
# #     update_endpoint=True
# )

In [ ]:
# estimator.delete_endpoint()